In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
import warnings; warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('lyrics_titles_AutoPump.csv')

In [4]:
all_lyrics = open('all_lyrics.txt', 'w')

In [5]:
data.head()

,Unnamed: 0,title,lyrics
0,0,1 Choppa 2 Choppa*,"[Intro]\r\nAyy, ayy\r\n\r\n[Chorus]\r\nOne cho..."
1,0,30's,"[Verse 1: Lil Pump]\r\nI'ma hit a stain, I'ma ..."
2,0,70 Nigga,\r\n Lyrics for this song...
3,0,Aight,"[Intro: Lil Pump]\r\nYuh, ouu, ouu, ouu, ouu, ..."
4,0,At the Door,"[Intro]\r\nOoh, Big Head on the beat\r\nLil Pu..."


In [6]:
for row in data.itertuples():
    text = row.lyrics
    all_lyrics.write(text)

In [7]:
all_lyrics.close()

In [19]:
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    if line.startswith('  '):
        print(line)

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Lil Pump]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Lil Pump]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Verse]          

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Rick Ross]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

   

In [23]:
lines = []
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    line = line.lstrip()
    if line.startswith('[Chorus'):
        lines.append('[Chorus]\n')
    elif line.startswith('[Verse'):
        lines.append('[Verse]\n')
    elif line.startswith('[Hook'):
        lines.append('[Hook]\n')
    elif line.startswith('[Intro'):
        lines.append('[Intro]\n')
    elif line.startswith('[Outro'):
        lines.append('[Outro]\n')
    elif line.startswith('[Bridge'):
        lines.append('[Bridge]\n')
    elif line.startswith('[Interlude'):
        lines.append('[Interlude]\n')
    elif line.startswith('Lyrics for this'):
        lines.append('\n')
    else:
        lines.append(line)
all_lyrics.close()

In [24]:
all_lyrics_cleaned = open('all_lyrics_cleaned.txt', 'w')
for line in lines:
    all_lyrics_cleaned.write(line)
all_lyrics_cleaned.close()   